In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Autoencoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(True))
        self.decoder = nn.Sequential(
            nn.Linear(hidden_size, input_size),
            nn.ReLU(True))

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
class VAE(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(True))
        self.mu = nn.Linear(hidden_size, latent_size)
        self.logvar = nn.Linear(hidden_size, latent_size)
        self.decoder = nn.Sequential(
            nn.Linear(latent_size, hidden_size),
            nn.ReLU(True),
            nn.Linear(hidden_size, input_size),
            nn.ReLU(True))

    def forward(self, x):
        x = self.encoder(x)
        mu = self.mu(x)
        logvar = self.logvar(x)
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        z = mu + eps*std
        x = self.decoder(z)
        return x, mu, logvar
    


In [ ]:


X = torch.randn(100, 784)
model = Autoencoder(input_size=784, hidden_size=400, latent_size=20)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
for epoch in range(100):
    X_hat = model(X)
    loss = F.mse_loss(X_hat, X)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

X = torch.randn(100, 784)
model = VAE(input_size=784, hidden_size=400, latent_size=20)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
for epoch in range(100):
    X_hat, mu, logvar = model(X)
    loss = F.mse_loss(X_hat, X) + 0.5 * torch.sum(torch.exp(logvar) + mu**2 - 1. - logvar)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
X = torch.randn(100, 784)
Y = torch.randn(100, 784)
model = GAN(input_size=784, hidden_size=400, latent_size=20)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
for epoch in range(100):
    X_hat, mu, logvar, D = model(X)

In [ ]:
#GAN model for 1D time series data, with the goal of predicting the next time step

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

# Define the model
class GAN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GAN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.input_size = input_size
        
        self.encoder = nn.Sequential(
            nn.Linear(self.input_size, self.hidden_size),
            nn.ReLU(),
            nn.Linear(self.hidden_size, self.output_size),
            nn.ReLU()
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(self.output_size, self.hidden_size),
            nn.ReLU(),
            nn.Linear(self.hidden_size, self.input_size),
            nn.ReLU()
        )
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
    def predict(self, x):
        x = self.encoder(x)
        return x
    
    def loss(self, x, y):
        x = self.forward(x)
        return F.mse_loss(x, y)
    
    def train(self, x, y, epochs=1000, lr=0.01):
        optimizer = optim.Adam(self.parameters(), lr=lr)
        for epoch in range(epochs):
            optimizer.zero_grad()
            loss = self.loss(x, y)
            loss.backward()
            optimizer.step()
            if epoch % 100 == 0:
                print('Epoch: {} Loss: {}'.format(epoch, loss.item()))